In [3]:
import pandas as pd

In [7]:
DATA_FOLDER = './data/'

In [8]:
df = pd.read_pickle(DATA_FOLDER + 'clean_pass_01p2_Ptarget_FPemp_ident.pkl')

In [9]:
def map_plot(negps, value_name ='value', v2=None  ,v2_name=''):    
    from bokeh.io import show
    from bokeh.models import LogColorMapper, LinearColorMapper, BasicTicker, ColorBar
    from bokeh.palettes import Viridis6 as palette
    from bokeh.plotting import figure
    from bokeh.sampledata.unemployment import data as unemployment
    from bokeh.sampledata.us_states import data as states
    
    palette = tuple(reversed(palette))
    
    s = {
        s: d for s, d in states.items() if s in negps.index
    }
    s_xs = [d["lons"] for d in s.values()]
    s_ys = [d["lats"] for d in s.values()]
    s_names = [states['name'] for states in s.values()]
    s_rates = [negps[s] for s,d in states.items() if s in negps.index]
    color_mapper = LogColorMapper(palette=palette) # LinearColorMapper(palette=palette)#
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker= BasicTicker(),
                         location=(0,0))
    
    data=dict(
        x=s_xs,
        y=s_ys,
        name=s_names,
        rate=s_rates,
    )
    
    TOOLS = "pan,box_zoom,reset,hover,save"
    
    p = figure(
        title="Landing Club - State Wise " + value_name, tools=TOOLS,
        x_axis_location=None, y_axis_location=None,
        tooltips=[
            ("Name", "@name"), (value_name , "@rate"), ("(Long, Lat)", "($x, $y)")
        ])
    p.grid.grid_line_color = None
    p.hover.point_policy = "follow_mouse"
    
    p.patches('x', 'y', source=data,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)
    
    p.add_layout(color_bar, 'right')
    #p.text()
    show(p)


In [10]:
df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'last_credit_pull_d',
       'pub_rec_bankruptcies', 'has_made_pymnt', 'target', 'emp_ident'],
      dtype='object')

In [11]:

gb = df.groupby('addr_state')
negps = gb.count().target - gb.sum().target
#negps = negps/negps.sum()
map_plot(negps, 'charge-off count')

In [30]:
negps = (gb.count().target - gb.sum().target) * (1/gb.count().target)
#negps = negps/negps.sum()
map_plot(negps, 'charge-off rate')

In [32]:
negps = gb.count().target
#negps = negps/negps.sum()
map_plot(negps, 'credit count')

In [34]:
negps = gb.mean().loan_amnt 
#negps = negps/negps.sum()
map_plot(negps, 'mean loan amount')